In [1]:
import logging
import sys
import os

import zipfile

import importlib

from six.moves import urllib
from shutil import copy2

import matplotlib.pyplot as plt

import cv2

import numpy as np

from skimage.segmentation import slic
from skimage.segmentation import mark_boundaries
from skimage.util import img_as_float

In [2]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K


Using TensorFlow backend.
C:\Users\Usrer\Anaconda3\envs\tf_gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Usrer\Anaconda3\envs\tf_gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Usrer\Anaconda3\envs\tf_gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Usrer\Anaconda3\envs\tf_gpu\lib\sit

In [3]:
batch_size = 128
num_classes = 2
epochs = 12

# input image dimensions
img_rows, img_cols = 48, 48

In [4]:
def download(url, dest_directory):
    filename = url.split('/')[-1]
    filepath = os.path.join(dest_directory, filename)

    logging.info("Download URL: {}".format(url))
    logging.info("Download DIR: {}".format(dest_directory))

    def _progress(count, block_size, total_size):
                prog = float(count * block_size) / float(total_size) * 100.0
                sys.stdout.write('\r>> Downloading %s %.1f%%' %
                                 (filename, prog))
                sys.stdout.flush()

    filepath, _ = urllib.request.urlretrieve(url, filepath,
                                             reporthook=_progress)
    print()
    return filepath


importlib.reload(logging)
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

# Download KITTI DATA

kitti_data_url = 'https://s3.eu-central-1.amazonaws.com/avg-kitti/data_road.zip'

data_dir = './data'

data_road_zip = './data/data_road.zip'


if not os.path.exists(data_road_zip):
    if kitti_data_url == '':
        logging.error("Data URL for Kitti Data not provided.")
        url = "http://www.cvlibs.net/download.php?file=data_road.zip"
        logging.error("Please visit: {}".format(url))
        logging.error("and request Kitti Download link.")
        logging.error("Rerun scipt using"
                      "'python download_data.py' --kitti_url [url]")
        exit(1)
    if not kitti_data_url[-19:] == 'kitti/data_road.zip':
        logging.error("Wrong url.")
        url = "http://www.cvlibs.net/download.php?file=data_road.zip"
        logging.error("Please visit: {}".format(url))
        logging.error("and request Kitti Download link.")
        logging.error("Rerun scipt using"
                      "'python download_data.py' --kitti_url [url]")
        exit(1)
    else:
        logging.info("Downloading Kitti Road Data.")
        download(kitti_data_url, data_dir)

# Extract and prepare KITTI DATA
logging.info("Extracting kitti_road data.")
zipfile.ZipFile(data_road_zip, 'r').extractall(data_dir)
kitti_road_dir = os.path.join(data_dir, 'data_road/')

#logging.info("Preparing kitti_road data.")

train_txt = "data/train3.txt"
val_txt = "data/val3.txt"
testing_txt = "data/testing.txt"

sys.stdout.flush()

#copy2(train_txt, kitti_road_dir)
#copy2(val_txt, kitti_road_dir)
#copy2(testing_txt, kitti_road_dir)

#logging.info("All data have been downloaded successful.")

INFO:root:Extracting kitti_road data.


In [5]:
def getData(txt):
    all_x = []
    all_y = []

    with open(txt, 'r') as f:
        for line in f:
            names = line.split()
            logging.info(names[0])
            #logging.info(names[1])        
            # load the image and apply SLIC and extract (approximately)

            # the supplied number of segments
            image = cv2.imread(os.path.join(kitti_road_dir, names[0]))
            segments = slic(img_as_float(image), n_segments = 200, sigma = 5)

            image2 = cv2.imread(os.path.join(kitti_road_dir, names[1]))
            magenta = np.array([255, 0, 255], dtype=np.uint8)
            #mask2 = np.zeros(image.shape[:2], dtype = "uint8")
            #mask2[image2 == magenta] = 255

            #data = np.array(image2)
            #rgb = data[:,:,:3]
            #mask2 = np.all(rgb == magenta, axis = -1)

            #lower_magenta = np.array([254,0,254])
            #upper_magenta = np.array([255,1,255])
            #mask2 = cv2.inRange(image2 , lower_magenta, upper_magenta)        

            # ok
            mask2 = np.all(image2 == magenta, axis = -1)
            #plt.imshow(mask2)

            # show the output of SLIC
            #fig = plt.figure("Superpixels")
            #ax = fig.add_subplot(1, 1, 1)
            #ax.imshow(mark_boundaries(img_as_float(cv2.cvtColor(image, cv2.COLOR_BGR2RGB)), segments))
            #plt.axis("off")
            #plt.show()
            # loop over the unique segment values
            for (i, segVal) in enumerate(np.unique(segments)):
                # construct a mask for the segment
                #print("[x] inspecting segment %d" % (i))

                mask = np.zeros(image.shape[:2], dtype = "uint8")
                mask[segments == segVal] = 255

                m = cv2.moments(mask, True);
                #print("centroid coordinates %d : %d" % (m["m10"]/m["m00"], m["m01"]/m["m00"]))

                x = m["m10"]/m["m00"]
                y = m["m01"]/m["m00"]

                intersection = np.logical_and(mask, mask2)


                gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                img = cv2.bitwise_and(gray, gray, mask = mask)

                #intersection_mask = intersection.astype(np.uint8)
                #intersection_mask *= 255
                #img2 = cv2.bitwise_and(gray, gray, mask = intersection_mask)
                #img2 = img2[int(y - img_rows/2) : int(y + img_rows/2), int(x - img_cols/2) : int(x + img_cols/2)]

                #crop
                img = img[int(y - img_rows/2) : int(y + img_rows/2), int(x - img_cols/2) : int(x + img_cols/2)]

                old_size = img.shape[:2]
                delta_w = img_cols - old_size[1]#img.cols
                delta_h = img_rows - old_size[0]#img.rows
                top, bottom = delta_h//2, delta_h-(delta_h//2)
                left, right = delta_w//2, delta_w-(delta_w//2)

                img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=0)


                value = int(np.sum(intersection) > m["m00"] / 2)
                #print("value %d" % (value))

                #all_x.append(np.asarray(img[:,:]))
                all_x.append([img])
                all_y.append([value])


                #print("value %d" % (value))

                #if intersection.any():
                #if m["m00"] == np.sum(intersection):
                #if np.sum(intersection) > m["m00"] / 2:

                # show the masked region
                #v2.imshow("Mask", mask)
                #v2.imshow("Applied", cv2.bitwise_and(image, image, mask = mask))

                    #print("centroid, intersection counts %d %d" % (m["m00"], np.sum(intersection)))

                    #m = cv2.moments(mask, True);
                    #print("centroid coordinates %d : %d" % (m["m10"]/m["m00"], m["m01"]/m["m00"]))

                    #img = cv2.bitwise_and(image, image, mask = mask)
                    #plt.imshow(img)
                    #plt.imshow(img)
                    #break
            #break

    all_x = np.concatenate(all_x)
    all_y = np.concatenate(all_y)

    #all_x = all_x.astype('float32')
    #print('all_x shape:', all_x.shape)
    #print(all_x)
    #print('all_y shape:', all_y.shape)

    return all_x, all_y


In [6]:
# the data, split between train and test sets
(x_train, y_train) = getData(train_txt)
(x_test, y_test) = getData(val_txt)


INFO:root:training/image_2/um_000000.png
INFO:root:training/image_2/um_000001.png
INFO:root:training/image_2/um_000002.png
INFO:root:training/image_2/um_000003.png
INFO:root:training/image_2/um_000004.png
INFO:root:training/image_2/um_000005.png
INFO:root:training/image_2/um_000006.png
INFO:root:training/image_2/um_000007.png
INFO:root:training/image_2/um_000008.png
INFO:root:training/image_2/um_000009.png
INFO:root:training/image_2/um_000010.png
INFO:root:training/image_2/um_000011.png
INFO:root:training/image_2/um_000012.png
INFO:root:training/image_2/um_000013.png
INFO:root:training/image_2/um_000014.png
INFO:root:training/image_2/um_000015.png
INFO:root:training/image_2/um_000016.png
INFO:root:training/image_2/um_000017.png
INFO:root:training/image_2/um_000018.png
INFO:root:training/image_2/um_000019.png
INFO:root:training/image_2/um_000020.png
INFO:root:training/image_2/um_000021.png
INFO:root:training/image_2/um_000022.png
INFO:root:training/image_2/um_000023.png
INFO:root:traini

INFO:root:training/image_2/uu_000039.png
INFO:root:training/image_2/uu_000040.png
INFO:root:training/image_2/uu_000041.png
INFO:root:training/image_2/uu_000042.png
INFO:root:training/image_2/uu_000043.png
INFO:root:training/image_2/uu_000044.png
INFO:root:training/image_2/uu_000045.png
INFO:root:training/image_2/uu_000046.png
INFO:root:training/image_2/uu_000047.png
INFO:root:training/image_2/uu_000048.png
INFO:root:training/image_2/uu_000049.png
INFO:root:training/image_2/uu_000050.png
INFO:root:training/image_2/uu_000051.png
INFO:root:training/image_2/uu_000052.png
INFO:root:training/image_2/uu_000053.png
INFO:root:training/image_2/uu_000054.png
INFO:root:training/image_2/uu_000055.png
INFO:root:training/image_2/uu_000056.png
INFO:root:training/image_2/uu_000057.png
INFO:root:training/image_2/uu_000058.png
INFO:root:training/image_2/uu_000059.png
INFO:root:training/image_2/uu_000060.png
INFO:root:training/image_2/uu_000061.png
INFO:root:training/image_2/uu_000062.png
INFO:root:traini

In [7]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(52, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

x_train shape: (43580, 48, 48, 1)
y_train shape: (43580,)
43580 train samples
8668 test samples


Train on 43580 samples, validate on 8668 samples
Epoch 1/12
43580/43580 [==============================] - 24s 554us/step - loss: 0.3674 - accuracy: 0.8377 - val_loss: 0.3278 - val_accuracy: 0.8546
Epoch 2/12
43580/43580 [==============================] - 20s 448us/step - loss: 0.2834 - accuracy: 0.8807 - val_loss: 0.2563 - val_accuracy: 0.8909
Epoch 3/12
43580/43580 [==============================] - 20s 448us/step - loss: 0.2493 - accuracy: 0.8898 - val_loss: 0.2431 - val_accuracy: 0.8936
Epoch 4/12
43580/43580 [==============================] - 20s 448us/step - loss: 0.2316 - accuracy: 0.8968 - val_loss: 0.2527 - val_accuracy: 0.8854
Epoch 5/12
43580/43580 [==============================] - 20s 448us/step - loss: 0.2164 - accuracy: 0.9051 - val_loss: 0.2702 - val_accuracy: 0.8910
Epoch 6/12
43580/43580 [==============================] - 20s 449us/step - loss: 0.2076 - accuracy: 0.9089 -